In [ ]:
%%capture
# @title 1. 라이브러리 설치
%pip install -q fastapi uvicorn[standard]
%pip install -q pyngrok>=7.0.0 diffusers>=0.27.0 transformers accelerate
%pip install -q peft controlnet_aux torch>=2.0.0 opencv-python-headless Pillow
%pip install -q requests safetensors pydantic huggingface_hub python-multipart

print("✅ 라이브러리 설치 완료")

In [ ]:
# @title 2. Hugging Face 로그인 / Ngrok 설정 (Authtoken 시크릿 키)
import os
from google.colab import userdata
from huggingface_hub import login
from pyngrok import conf, ngrok

# Hugging Face 및 ngrok 토큰 (Colab Secret에서 가져오기)
hf_token = userdata.get("HF_TOKEN")        # Hugging Face Token
ngrok_token = userdata.get("NGROK_TOKEN")  # ngrok Token

if hf_token == None:
  print('x')
if ngrok_token == None:
  print('x')

# 포트 설정
PORT = 8000

# hugging face 로그인
print("🔑 Hugging Face 로그인 중...")
login(token=hf_token)

# ngrok 설정
if ngrok_token == "YOUR_NGROK_AUTHTOKEN":
  print("⚠️ 경고: Ngrok Authtoken을 입력하세요. 없으면 Ngrok 터널이 작동하지 않습니다.")
  print("Ngrok 토큰은 https://dashboard.ngrok.com/get-started/your-authtoken 에서 확인 가능합니다.")
else:
  os.environ['NGROK_AUTHTOKEN'] = ngrok_token
  conf.get_default().auth_token = ngrok_token
  print("✅ Ngrok Authtoken 설정 완료")

🔑 Hugging Face 로그인 중...
✅ Ngrok Authtoken 설정 완료


In [ ]:
# @title 3. FastAPI 애플리케이션 코드 작성 (main_server.py 파일 생성)
# 이 셀은 FastAPI 서버 코드를 main_server.py 파일로 저장합니다.

%%writefile main_server.py
import os
import io
import logging
from contextlib import asynccontextmanager
from typing import Optional, Any

import torch
import uvicorn
from fastapi import FastAPI, HTTPException, UploadFile, File, Form
from fastapi.responses import Response
from PIL import Image
from diffusers import FluxControlNetModel, FluxControlNetPipeline
from controlnet_aux import CannyDetector

# --- Configuration ---
BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev"
CONTROLNET_MODEL_ID = "Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0"
DEFAULT_STEPS = 30
DEFAULT_GUIDANCE_SCALE = 3.5
DEFAULT_CONTROLNET_SCALE = 0.7
DEFAULT_LORA_SCALE = 0.8
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024
PORT = 8000

# --- Global State ---
controlnet_pipe: Optional[FluxControlNetPipeline] = None
controlnet_preprocessor: Optional[Any] = None
device: Optional[str] = None

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Helper Functions ---
def load_pil_image(image_bytes: bytes) -> Image.Image:
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    return image

def image_to_bytes(image: Image.Image) -> bytes:
    byte_arr = io.BytesIO()
    image.save(byte_arr, format='PNG')
    byte_arr.seek(0)
    return byte_arr.getvalue()

def get_generator(seed: Optional[int] = None) -> torch.Generator:
    if seed is None:
        seed = torch.randint(0, 2**32 - 1, (1,)).item()
    logger.info(f"Using seed: {seed}")
    return torch.Generator(device=device).manual_seed(seed)

def prepare_control_image(uploaded_image: UploadFile) -> Image.Image:
    if controlnet_preprocessor is None:
        raise RuntimeError("ControlNet preprocessor not loaded.")
    image = load_pil_image(uploaded_image.file.read())
    control_image = controlnet_preprocessor(image)
    return control_image

# --- Model Loading ---
def load_models():
    global controlnet_pipe, controlnet_preprocessor, device

    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    dtype = torch.bfloat16 if device == "cuda" and torch.cuda.is_bf16_supported() else torch.float16

    try:
        logger.info("Loading ControlNet model...")
        controlnet_model = FluxControlNetModel.from_pretrained(CONTROLNET_MODEL_ID, torch_dtype=dtype)
        controlnet_pipe = FluxControlNetPipeline.from_pretrained(BASE_MODEL_ID, controlnet=controlnet_model, torch_dtype=dtype)
        controlnet_pipe.to(device)
        controlnet_pipe.enable_model_cpu_offload()
        controlnet_pipe.enable_attention_slicing()

        logger.info("Loading Canny preprocessor...")
        controlnet_preprocessor = CannyDetector()
        if hasattr(controlnet_preprocessor, 'to'):
            controlnet_preprocessor.to(device)

    except Exception as e:
        logger.exception("Fatal error during model loading")
        raise RuntimeError(f"Failed to load models: {e}")

    logger.info("✅ Model loading complete.")

# --- FastAPI Setup ---
@asynccontextmanager
async def lifespan(app: FastAPI):
    logger.info("Application startup...")
    load_models()
    yield
    logger.info("Application shutdown...")
    global controlnet_pipe
    del controlnet_pipe
    if device == 'cuda':
        torch.cuda.empty_cache()

app = FastAPI(lifespan=lifespan, title="Flux ControlNet Image-to-Image API")

# --- API Endpoint ---
@app.post("/generate/image-to-image", summary="Generate Image using ControlNet", response_class=Response)
async def generate_image_to_image(
    prompt: str = Form(...),
    negative_prompt: Optional[str] = Form(""),
    lora_scale: Optional[float] = Form(DEFAULT_LORA_SCALE),
    controlnet_scale: float = Form(DEFAULT_CONTROLNET_SCALE),
    num_inference_steps: int = Form(DEFAULT_STEPS),
    guidance_scale: float = Form(DEFAULT_GUIDANCE_SCALE),
    seed: Optional[int] = Form(None),
    image: UploadFile = File(...)
):
    if controlnet_pipe is None:
        raise HTTPException(status_code=503, detail="ControlNet pipeline not ready.")

    control_image = prepare_control_image(image)

    generator = get_generator(seed)

    try:
        with torch.inference_mode():
            result = controlnet_pipe(
                prompt=prompt,
                control_image=control_image,  # ✨ 주의: FluxControlNetPipeline은 control_image를 받을 것
                width=IMAGE_WIDTH,
                height=IMAGE_HEIGHT,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                controlnet_conditioning_scale=controlnet_scale,
                generator=generator,
            )

        if hasattr(result, 'images'):
            output_image = result.images[0]
        elif isinstance(result, list) and isinstance(result[0], Image.Image):
            output_image = result[0]
        else:
            raise ValueError("Could not extract image from pipeline result.")

        img_bytes = image_to_bytes(output_image)
        return Response(content=img_bytes, media_type="image/png")

    except Exception as e:
        logger.exception("Image-to-Image generation failed")
        raise HTTPException(status_code=500, detail=f"Generation failed: {e}")

# --- Main ---
if __name__ == "__main__":
    logger.info("Starting Uvicorn server...")
    uvicorn.run("main_server:app", host="0.0.0.0", port=PORT, reload=False)


Writing main_server.py


In [ ]:
# @title 4. ngrok 터널링 및 api 제공
# 환경 변수 설정
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import time
import os
import subprocess
from pyngrok import ngrok
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
pyngrok_logger = logging.getLogger("pyngrok")
pyngrok_logger.setLevel(logging.INFO)

# --- 설정 ---
LOG_FILE = "uvicorn_server.log"
PORT = 8000  # main_server.py 내부 포트와 일치
STATIC_NGROK_DOMAIN = "publicly-capable-monkfish.ngrok-free.app"  # 고정 도메인

# --- 기존 서버 및 ngrok 프로세스 종료 ---
print("ℹ️ 기존 Uvicorn/Ngrok 프로세스 종료 시도...")

try:
    ngrok.kill()
    print("   - ngrok 프로세스 종료 완료 (pyngrok).")
    time.sleep(2)
except Exception as e:
    pyngrok_logger.warning(f"ngrok.kill() 실행 중 오류 발생 (무시 가능): {e}")

subprocess.run(['pkill', '-f', 'uvicorn main_server:app'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', f'ngrok.*http.*{PORT}'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', '/root/.config/ngrok/ngrok'], stderr=subprocess.DEVNULL)
time.sleep(3)

# --- FastAPI 서버 백그라운드 실행 ---
print(f"🚀 FastAPI 서버를 백그라운드에서 시작합니다... 로그 파일: {LOG_FILE}")
nohup_cmd = f"nohup python main_server.py > {LOG_FILE} 2>&1 &"
subprocess.Popen(nohup_cmd, shell=True)
time.sleep(5)  # 서버 초기 부팅 대기

# --- ngrok static 도메인으로 연결 ---
print(f"🌐 ngrok static domain 연결 시도 ({STATIC_NGROK_DOMAIN})...")
public_url = ngrok.connect(
    addr=PORT,
    proto="http",
    domain=STATIC_NGROK_DOMAIN
)
print(f"✅ 고정 URL 연결 완료: {public_url}")

# --- 서버 준비 대기 (모델 로딩 시간 확보) ---
wait_seconds = 200  # 필요에 따라 조정
print(f"⏳ 서버 및 모델 준비 대기 중 ({wait_seconds}초)...")
for i in range(wait_seconds):
    print(str(i), end=" ", flush=True)
    if (i + 1) % 30 == 0:
        print()
    time.sleep(1)
print("\n✅ 서버 준비 대기 완료.")

# --- 결과 출력 ---
print(f"\n🎯 서버가 백그라운드에서 실행되고 있으며 외부 접속 URL은 다음과 같습니다:")
print(f"🔗 {public_url}")

print("\n--- 실행 중인 관련 프로세스 (참고용) ---")
!ps -ef | grep -E "main_server.py|ngrok" | grep -v -E "grep|pkill|colab"


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
ℹ️ 기존 Uvicorn/Ngrok 프로세스 종료 시도...
   - ngrok 프로세스 종료 완료 (pyngrok).
🚀 FastAPI 서버를 백그라운드에서 시작합니다... 로그 파일: uvicorn_server.log


INFO:pyngrok.ngrok:Opening tunnel named: http-8000-dce04423-330f-4cb6-8795-06f1757b8d37


🌐 ngrok static domain 연결 시도 (publicly-capable-monkfish.ngrok-free.app)...


INFO:pyngrok.process:Overriding default auth token
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="client session established" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg=start pg=/api/tunnels id=7d2243fd67d7674d
INFO:pyngrok.process.ngrok:t=2025-04-23T10:45:14+0000 lvl=info msg=end pg=/api/tunnels id=7d2243fd67d7

✅ 고정 URL 연결 완료: NgrokTunnel: "https://publicly-capable-monkfish.ngrok-free.app" -> "http://localhost:8000"
⏳ 서버 및 모델 준비 대기 중 (200초)...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 
180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
✅ 서버 준비 대기 완료.

🎯 서버가 백그라운드에서 실행되고 있으며 외부 접속 URL은 다음과 같습니다:
🔗 NgrokTunnel: "https://publicly-capable-monkfish.ngrok-free.app" -> "http://localhost:8000"

--- 실행 중인 관련 프

In [ ]:
# # @title 5. API 서버 요청, CSV 기반 다수 프롬프트 사용 및 Google Drive 저장
# import requests
# import base64
# import io
# from PIL import Image
# from IPython.display import display, HTML
# import time
# import json
# import os # os 모듈 추가
# import pandas as pd # pandas 라이브러리 추가
# from google.colab import drive # Google Drive 연동용

# # --- Google Drive 마운트 ---
# try:
#     drive.mount('/content/drive')
#     drive_mounted = True
#     print("✅ Google Drive 마운트 성공!")
# except Exception as e:
#     print(f"⚠️ Google Drive 마운트 실패: {e}")
#     print("   이미지는 Google Drive에 저장되지 않습니다.")
#     drive_mounted = False

# # --- 설정 ---
# # 5번 셀 출력에서 확인한 Ngrok URL (이전 성공 시 사용한 URL 유지)
# NGROK_URL = "https://7178-34-139-109-175.ngrok-free.app" # 이전 성공 시 사용한 URL

# # --- Google Drive 저장 경로 설정 ---
# SAVE_DIR = "/content/drive/MyDrive/FluxComicOutput" # 원하는 경로로 수정하세요.
# if drive_mounted:
#     try:
#         os.makedirs(SAVE_DIR, exist_ok=True)
#         print(f"✅ 이미지를 저장할 경로: {SAVE_DIR}")
#     except Exception as e:
#         print(f"⚠️ Google Drive 저장 경로 생성 실패 ({SAVE_DIR}): {e}")
#         print("   이미지는 Google Drive에 저장되지 않습니다.")
#         drive_mounted = False

# # --- CSV에서 프롬프트 로드 ---
# CSV_FILE_PATH = 'news_based_comic_prompts_sample.csv' # 업로드한 CSV 파일 이름

# # --- 생성할 Case ID 리스트 ---
# # 아래 리스트에 생성하고 싶은 만화의 case_id 번호들을 넣으세요.
# # 예: [1, 5, 10] 또는 list(range(1, 6)) -> 1번부터 5번까지 생성
# case_ids_to_generate = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # <--- 생성할 case_id 리스트를 여기에 지정하세요.

# try:
#     df_all_panels = pd.read_csv(CSV_FILE_PATH) # 전체 CSV 파일을 한번만 읽음
#     print(f"✅ '{CSV_FILE_PATH}' 파일 로드 성공!")
#     csv_loaded = True
# except FileNotFoundError:
#     print(f"❌ 오류: '{CSV_FILE_PATH}' 파일을 찾을 수 없습니다. 파일을 Colab 환경에 업로드했는지 확인하세요.")
#     csv_loaded = False
# except Exception as e:
#     print(f"❌ CSV 파일 처리 중 오류 발생: {e}")
#     csv_loaded = False
# # --------------------------

# if NGROK_URL == "YOUR_NGROK_PUBLIC_URL":
#     print("⚠️ 경고: NGROK_URL 변수에 5번 셀에서 얻은 실제 Ngrok 주소를 입력해야 합니다!")
# elif not csv_loaded: # CSV 로드 실패 시 실행 중지
#      print("❌ CSV 파일 로드 실패로 이미지 생성을 진행할 수 없습니다.")
# else:
#     # API 엔드포인트 주소 설정
#     TEXT_TO_IMAGE_URL = f"{NGROK_URL}/generate/text-to-image"
#     # IMAGE_TO_IMAGE_URL = f"{NGROK_URL}/generate/image-to-image" # 현재 비활성화 상태
#     CHANGE_LORA_URL = f"{NGROK_URL}/change-lora"
#     STATUS_URL = f"{NGROK_URL}/"

#     # --- 기본 생성 파라미터 ---
#     negative_prompt = "(worst quality, low quality, normal quality:1.2), deformed, blurry, text, signature"
#     num_inference_steps = 30
#     guidance_scale = 3.5
#     lora_scale_vintage = 0.0 # LoRA 사용 시 값 조정

#     # --- 서버 상태 확인 (전체 실행 전 한 번만) ---
#     print(f"\nAPI 서버 ({NGROK_URL}) 상태 확인 중...")
#     server_ok = False
#     try:
#         status_response = requests.get(STATUS_URL, timeout=30)
#         status_response.raise_for_status()
#         print("\n--- 서버 상태 ---")
#         print(json.dumps(status_response.json(), indent=2))
#         print("------------------\n")
#         server_ok = True
#     except requests.exceptions.RequestException as e:
#         print(f"❌ 서버 상태 확인 실패: {e}. 이미지 생성을 진행할 수 없습니다.")
#     # ------------------------------------

#     # --- 생성 실행 (지정된 모든 Case ID에 대해 반복) ---
#     if server_ok and csv_loaded:
#         print(f"🚀 총 {len(case_ids_to_generate)}개의 만화 생성을 시작합니다: {case_ids_to_generate}")

#         # 지정된 모든 case_id에 대해 외부 루프 실행
#         for selected_case_id in case_ids_to_generate:
#             print(f"\n===================================")
#             print(f"⏳ Case ID {selected_case_id} 처리 시작...")
#             print(f"===================================")

#             generated_images = [] # 각 케이스별 이미지 리스트 초기화 (메모리 관리)

#             # --- 해당 Case ID의 패널 데이터 준비 ---
#             selected_panels_df = df_all_panels[df_all_panels['case_id'] == selected_case_id].sort_values(by='panel')

#             if selected_panels_df.empty:
#                 print(f"❌ 오류: CSV 파일에서 case_id {selected_case_id}를 찾을 수 없습니다. 다음 case_id로 넘어갑니다.")
#                 continue # 현재 case_id 건너뛰고 다음 루프 반복 실행

#             comic_panels = selected_panels_df[['prompt', 'seed']].to_dict('records')
#             print(f"✅ Case ID {selected_case_id}에 대한 패널 데이터 로드 완료 ({len(comic_panels)}개 패널).")
#             # ------------------------------------

#             # --- 각 패널 이미지 생성 (내부 루프) ---
#             for i, panel_data in enumerate(comic_panels):
#                 if 'prompt' not in panel_data or 'seed' not in panel_data:
#                      print(f"⚠️ 컷 {i+1} 데이터 형식 오류. 건너<0xEB><0x9C><0x84>니다: {panel_data}")
#                      continue

#                 panel_prompt = panel_data['prompt']
#                 panel_seed = int(panel_data['seed'])

#                 print(f"  컷 {i+1}/{len(comic_panels)} 생성 중: \"{panel_prompt[:50]}...\" (Seed: {panel_seed})")

#                 payload = {
#                     "prompt": panel_prompt,
#                     "negative_prompt": negative_prompt,
#                     "num_inference_steps": num_inference_steps,
#                     "guidance_scale": guidance_scale,
#                     "lora_scale": lora_scale_vintage,
#                     "seed": panel_seed
#                 }

#                 try:
#                     start_req_time = time.time()
#                     response = requests.post(TEXT_TO_IMAGE_URL, json=payload, timeout=300) # 타임아웃 늘림
#                     end_req_time = time.time()

#                     if response.status_code == 200:
#                         print(f"    ✅ 컷 {i+1} 생성 성공! (소요 시간: {end_req_time - start_req_time:.2f}초)")
#                         try:
#                             img = Image.open(io.BytesIO(response.content))
#                             generated_images.append(img)
#                             display(img)

#                             # --- Google Drive 저장 로직 ---
#                             if drive_mounted:
#                                 try:
#                                     filename = f"comic_case_{selected_case_id}_panel_{i+1}_seed_{panel_seed}.png"
#                                     save_path = os.path.join(SAVE_DIR, filename)
#                                     img.save(save_path)
#                                     print(f"      💾 이미지를 Google Drive에 저장했습니다: {save_path}")
#                                 except Exception as e:
#                                     print(f"      ⚠️ Google Drive 저장 실패 ({filename}): {e}")
#                             # ----------------------------

#                         except Exception as e:
#                             print(f"    ⚠️ 이미지를 표시/처리하는 중 오류: {e}")
#                     else:
#                         print(f"    ❌ 컷 {i+1} 생성 실패: Status Code {response.status_code}")
#                         try:
#                             print(f"       오류 메시지: {response.json()}")
#                         except json.JSONDecodeError:
#                             print(f"       오류 내용: {response.text}")
#                 except requests.exceptions.Timeout:
#                      print(f"  ❌ 컷 {i+1} 요청 시간 초과 (Timeout). 다음 패널로 넘어갑니다.")
#                 except requests.exceptions.RequestException as e:
#                      print(f"  ❌ 컷 {i+1} 요청 실패: {e}")
#                 except Exception as e:
#                      print(f"  ❌ 컷 {i+1} 처리 중 예상치 못한 오류: {e}")

#                 time.sleep(1) # 패널 간 딜레이

#             print(f"\n🎉 Case ID {selected_case_id} 만화 이미지 생성 완료!")
#             # time.sleep(5) # 필요하다면 케이스 사이에 추가 딜레이

#         print(f"\n===================================")
#         print(f"✅ 모든 요청된 만화 생성 작업 완료!")
#         print(f"===================================")
#         # --- 생성 실행 (Outer Loop for Case IDs) --- 끝 ---

#     # --- (선택 사항) 다른 LoRA 로 스타일 변경 후 재생성 ---
#     # (이 부분은 그대로 유지)